# INITALIZE

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DataType
from pyspark.sql.functions import col, trim, length

In [0]:
# Map the raw CRM column names for cleaner Silver Column names
RENAME_MAP = {
    "sls_ord_num": "order_number",
    "sls_prd_key": "product_key",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}

# Read Bronze Table

In [0]:
# Create a DataFrame that reads from the Bronze table
df = spark.table("workspace.bronze.crm_sales_details_raw")

# Silver Transformations

## Trimming

In [0]:
# For trimming the whitespace on all the string columents which helps to prevent filter and join issues from messy text
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

## Cleaning Dates

In [0]:
# Clean the date fields stored as yyyymmdd and set the invalid formats to null
df = (
    df
    .withColumn(
        "sls_order_dt",
        F.when(
            (col("sls_order_dt") == 0) | (length(col("sls_order_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_order_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_ship_dt",
        F.when(
            (col("sls_ship_dt") == 0) | (length(col("sls_ship_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_due_dt",
        F.when(
            (col("sls_due_dt") == 0) | (length(col("sls_due_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )
)

## Sales and Price Corrections

In [0]:
# Fux missing and invalid price by using sales / quantity when possible
df = (
    df
    .withColumn(
        "sls_price",
        F.when(
            (col("sls_price").isNull()) | (col("sls_price") <= 0),
            F.when(
                col("sls_quantity") != 0,
                col("sls_sales") / col("sls_quantity")
            ).otherwise(None)
        ).otherwise(col("sls_price"))
    )
)

## Renaming

In [0]:
# Rename the columns using the mapping from raw to standardized
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

# Writing the Silver Table

In [0]:
# We write a silver table that is cleaned + standardized
(
    df.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("silver.crm_sales")
)

In [0]:
%sql
-- Quick check
SELECT *
FROM silver.crm_sales